# Process Diverse(TM) Models

We've gone and run a lot of publicly available models on our [timaeus/pile-subsets-mini](https://huggingface.co/datasets/timaeus/pile_subsets_mini).

To help establish the link between fine grained pile-subset-losses and eval scores, we're looking at the structure of the latent factors 
Or latent factors thereof....


In [ ]:
import os
import pickle
import numpy as np
import pandas as pd
import os
import lsoc
from lsoc.factor import factor, selection, vis, data
from sklearn.preprocessing import StandardScaler
import plotly.graph_objects as go  # temporarily don't drop
import plotly.express as px
from plotly.subplots import make_subplots
from lsoc.powerlaw.vis import assign_cols
# Get the path to the output directory
SAVE = False
save_path = "plots"

if SAVE:
    os.makedirs(save_path, exist_ok=True)

In [ ]:
fname = f"{data.default_path}/evals/psych-20.csv"
psych = pd.read_csv(fname, index_col=0)
psych.drop(index="Gemma_Instruct_(7B)", inplace=True)
assert psych.isna().sum().sum() == 0
psych.head()

In [ ]:
#scaler = StandardScaler()
scaler = factor.ConstScaler(100.)

X = pd.DataFrame(
    scaler.fit_transform(psych),
    columns=psych.columns,
    index=psych.index,
)

X.head()

In [ ]:
# Cross validate number of dimensions
model = factor.FA()  # or PCA
errs = selection.cross_validate(X, model, max_factors=7, n_folds=5, repeats=5)
fig = vis.crossval(*errs, method_name=model.name)
fig.show()

In [ ]:
if SAVE:
    fig.write_image(f"{save_path}/FA_crossval.pdf")

In [ ]:
from sklearn.decomposition import PCA
from sklearn.decomposition import FactorAnalysis
from factor_analyzer import FactorAnalyzer

In [ ]:
n_components = 4

# Both oblimin and varimax try to make the loadings sparse
# oblimin is oblique, allowing it to be *more sparse by having non-orthogonal scores
fa_final, fa_name = FactorAnalyzer(rotation='oblimin', n_factors=n_components), "FactorAnalyzer"  

def get_scores(X, fa):
    H = fa.loadings_
    X_scaled = (X - fa_final.mean_) / fa_final.std_
    return X_scaled @ H @ np.linalg.pinv(H.T @ H)

fa_final.fit(X)

scores = get_scores(X, fa_final)

if hasattr(fa_final, "components_"):
    print("using scikit components")
    loadings = fa_final.components_.T
else:
    print("using factoranalyzer components")
    loadings = fa_final.loadings_
    
component_names = [f"PC{i+1}" for i in range(n_components)]

In [ ]:
scale_loadings = True

# This code takes the loadings, makes a df H and plots it
H = loadings

if scale_loadings:
    H = H / np.abs(H).max(axis=0)
    extra = " (scaled range)"
else:
    extra = ""

H = pd.DataFrame(
    data=H,
    index=X.columns,
    columns=component_names,
)

fig = vis.heatmap(
    H,
    title=f"Task Loadings ({fa_name}){extra}",
    width=6,
    height=64,
    reversescale=True
)
fig.show()

In [ ]:
# Aggregate even further to subsets!
I = H.copy()
I['source'] = [s.split()[0] for s in H.index]
grouped_means = I.groupby('source').mean().T
grouped_means.round(3).to_dict(orient='list')


In [ ]:
# The Loadings are so tall that to put them in a report we should save them in parts:
if SAVE:
    fig1 = vis.heatmap(
        H[:80],
        title=f"Task Loadings ({fa_name}){extra}",
        width=6,
        height=32,
        reversescale=True,
        zmin=-1.,
        zmax=1.,
        showscale=False,
    )
    fig2 = vis.heatmap(
        H[80:160],
        title=f"Task Loadings ({fa_name}){extra}",
        width=6,
        height=32,
        reversescale=True,
        zmin=-1.,
        zmax=1.,
        showscale=False,
    )
    fig3 = vis.heatmap(
        H[160:],
        title=f"Task Loadings ({fa_name}){extra}",
        width=6,
        height=24,
        reversescale=True,
        zmin=-1.,
        zmax=1.,
        showscale=True,
    )

    
    fig1.write_image(f"{save_path}/loadings-a.png", scale=2)
    fig2.write_image(f"{save_path}/loadings-b.png", scale=2)
    fig3.write_image(f"{save_path}/loadings-c.png", scale=2)


In [ ]:
# Now display the model scores
scale_scores = True
# This code takes the loadings, makes a df W and plots it

W = -get_scores(X, fa_final).values  # Higher losses are worse (unlike accuracy)

if scale_scores:
    W = W / np.abs(W).max(axis=0)
    extra = " (scaled range)"
else:
    W = W + 0  # copy
    extra = ""

# Convert to dataframe
W = pd.DataFrame(
    data=W,
    index=X.index,
    columns=component_names,
)

# Sort by model name
W = W.sort_index(ascending=True)

fig = vis.heatmap(
    W, # positive loss is bad though?
    title=f"Scores ({fa_name}){extra}",
    width=8,
    height=12,
    reversescale=True
)

if SAVE:
    fig.write_image(f"{save_path}/scores.png", scale=2)
fig.show()

# Project Pythia Scores onto these factors

# Losses look a bit big for untrained models, but mostly... fine?


In [ ]:
fname = f"{data.default_path}/evals/pythia-psych-20.csv"
pythia = pd.read_csv(fname, index_col=0)
p_scaled = scaler.transform(pythia)
p_scaled.loc["Pythia_70m_143000"]

In [ ]:
X.loc["Pythia_(70m)"]  # YAY!!!

In [ ]:
W.loc["Pythia_(70m)"]

In [ ]:
"""Solves for W in the equation X = WH"""
ref_scores = get_scores(X, fa_final)
p_scores = get_scores(p_scaled, fa_final)
# _mu = ref_scores.mean(axis=0)
# _std = ref_scores.std(axis=0)
# r_scores = -(p_scores - _mu)/_std
# ref_scores = -(ref_scores - _mu)/_std

ma = ref_scores.abs().max(axis=0)
r_scores = -p_scores / ma
ref_scores = -ref_scores / ma


In [ ]:
ref_scores

In [ ]:
# Load the aggregated pythia features
pythia = ["70m", "160m", "410m", "1b", "1.4b", "2.8b", "6.9b"]  # "14m", "31m", 
available = p_scores.index.values
colors = assign_cols(list(pythia) + ["yellow"] , 'viridis')
compares = ref_scores.idxmax()  #list(set(ref_scores.idxmax()) | set(ref_scores.idxmin()))
ccols = assign_cols(compares, 'jet')
ymin = -1
# Create the figure with 4 stacked rows
fig = make_subplots(rows=n_components, cols=1, shared_xaxes=True, 
                    vertical_spacing=0.05, 
                    subplot_titles=component_names)


cutoff = 512


for size in pythia:
    
    inds = [a for a in available if size in a]
    if len(inds) == 0:
        continue
    steps = [int(v.split("_")[-1]) for v in inds]
    steps, inds = zip(*sorted(zip(steps, inds)))

    cut = np.argmax(np.array(steps)>cutoff)
    steps = steps[cut:]
    inds = inds[cut:]
    
    vals = r_scores.loc[list(inds)]
    name = "Pythia " + size
    
    # Add each component as a separate subplot in its own row
    for i in range(n_components):
        fig.add_trace(
            go.Scatter(x=steps, y=vals.loc[:, i], mode='lines', line=dict(color=colors[size]),
                       name=name, showlegend=i==0),
            row=i+1, col=1,
        )

for i in range(n_components):
    for cname in compares:
        v = ref_scores.loc[cname][i]
        fig.add_trace(
            go.Scatter(x=[steps[0], steps[-1]], y=[v, v], mode='lines', line=dict(color=ccols[cname]),  
                       name=cname, showlegend=i==0),
            row=i+1, col=1,
        )
    
# Update the layout with appropriate height based on number of subplots
fig.update_layout(
    height=250 * n_components,  # 250px per subplot
    width=900,
    showlegend=False,
    margin=dict(l=50, r=50, t=50, b=50)
)

# Update y-axis titles with component names
for i in range(n_components):
    fig.update_yaxes(title_text="Scaled Model Score", row=i+1, col=1)  # range=[ymin, 2], 
    fig.update_xaxes(type="log", row=i+1, col=1)
    
# Update x-axis title (only needed for bottom subplot)
fig.update_xaxes(title_text="Steps", row=n_components, col=1)
fig.update_layout(
    width=1000,
    height=2000,
    showlegend=True
)
# fig.update_layout
fig.write_image("/home/areid/newplots/traces.png", scale=2)
#fig.write_image("/home/areid/traces.png", scale=2)
fig.write_html("/home/areid/traces.html")
fig.show()

Copyright (c) Gradient Institute and Timaeus.

Licensed under the Apache 2.0 License.